<a href="https://colab.research.google.com/github/Abhinav4279/rnn-lstm/blob/electricity-consumption/Val_Final_Combined_domain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.8 MB/s eta 0:00:00


In [3]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import callbacks
from kerastuner.tuners import RandomSearch

df0 = pd.read_csv('/content/drive/MyDrive/Major Project - II/0_interpolate_n.csv')
df1 = pd.read_csv('/content/drive/MyDrive/Major Project - II/1_interpolate_n.csv')

<ipython-input-3-4ee72f508c91>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch
<ipython-input-3-4ee72f508c91>:10: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221

In [4]:
df0 = df0.iloc[:,2:]
df1 = df1.iloc[:3579,2:]
df0 = df0.drop(df0.index[0])
df1 = df1.drop(df1.index[0])

df0 = df0.fillna(0)
df0 = df0.astype('float64')
df1 = df1.fillna(0)
df1 = df1.astype('float64')

from sklearn.utils import resample
import pandas as pd
import numpy as np

# Randomly select the rows
n_rows=len(df0)-len(df1)
df2 = df0.sample(n=n_rows)

# Define a random multiplication factor between 0 and 0.8
mult_factor = np.random.uniform(low=0.2, high=0.8, size=(n_rows, df0.shape[1]))

# Multiply each column of the selected rows with the multiplication factor
df_modified = df2 * mult_factor
df_modified['1'] = 1.0

# Add the modified rows to a new dataframe
df1 = df1.append(df_modified)
# df1 = resample(df1,
#              replace=True,
#              n_samples=len(df0),
#              random_state=42)

df = pd.concat([df0, df1], ignore_index=True)

<ipython-input-4-6b93788883a6>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df_modified)


In [5]:
print(len(df1), len(df0))

36679 36679


In [6]:
# # Importing required libraries
# from sklearn.model_selection import train_test_split
# from scipy.interpolate import pchip_interpolate

# # dataset shuffling
# df = df.sample(frac=1).reset_index(drop=True)

# # split the dataset into training, validation, and test datasets
# trainvalX, testX, trainvaly, testy = train_test_split(df.iloc[:, 1:], df.iloc[:, :1], test_size=0.2, random_state=42)
# trainX, valX, trainy, valy = train_test_split(trainvalX, trainvaly, test_size=0.2, random_state=42)

# # Extract the statistical features from the data
# train_std_dev = np.std(trainX, axis=1)
# train_mean = np.mean(trainX, axis=1)
# train_median = np.median(trainX, axis=1)
# train_datamean = np.mean(trainX, axis=1)
# train_cfpmean = np.mean(trainX, axis=1)

# val_std_dev = np.std(valX, axis=1)
# val_mean = np.mean(valX, axis=1)
# val_median = np.median(valX, axis=1)
# val_datamean = np.mean(valX, axis=1)
# val_cfpmean = np.mean(valX, axis=1)

# test_std_dev = np.std(testX, axis=1)
# test_mean = np.mean(testX, axis=1)
# test_median = np.median(testX, axis=1)
# test_datamean = np.mean(testX, axis=1)
# test_cfpmean = np.mean(testX, axis=1)

# # Add time domain features
# train_stdsProb = np.sum(np.abs(trainX - train_mean.values.reshape(-1, 1)), axis=1) / trainX.shape[1]
# val_stdsProb = np.sum(np.abs(valX - val_mean.values.reshape(-1, 1)), axis=1) / valX.shape[1]
# test_stdsProb = np.sum(np.abs(testX - test_mean.values.reshape(-1, 1)), axis=1) / testX.shape[1]

# train_stdsMean = train_std_dev / train_mean
# val_stdsMean = val_std_dev / val_mean
# test_stdsMean = test_std_dev / test_mean

# train_outsprob = np.sum((trainX - train_mean.values.reshape(-1, 1)) ** 2, axis=1) / trainX.shape[1]
# val_outsprob = np.sum((valX - val_mean.values.reshape(-1, 1)) ** 2, axis=1) / valX.shape[1]
# test_outsprob = np.sum((testX - test_mean.values.reshape(-1, 1)) ** 2, axis=1) / testX.shape[1]


# train_outsDev = train_std_dev / train_median
# val_outsDev = val_std_dev / val_median
# test_outsDev = test_std_dev / test_median

# trainX = np.stack([train_std_dev, train_mean, train_median, train_datamean,  train_cfpmean, train_stdsProb, train_stdsMean, train_outsprob, train_outsDev], axis=1)
# testX = np.stack([test_std_dev, test_mean, test_median, test_datamean, test_cfpmean, test_stdsProb, test_stdsMean, test_outsprob, test_outsDev], axis=1)
# valX = np.stack([val_std_dev, val_mean, val_median, val_datamean, val_cfpmean, val_stdsProb, val_stdsMean, val_outsprob, val_outsDev], axis=1)

In [7]:
from sklearn.model_selection import train_test_split

df = df.sample(frac=1).reset_index(drop=True)

trainvalX, testX, trainvaly, testy = train_test_split(df.iloc[:, 1:], df.iloc[:, :1], test_size=0.2, random_state=42)
trainX, valX, trainy, valy = train_test_split(trainvalX, trainvaly, test_size=0.2, random_state=42)

# Extract the statistical features from the data
train_std_dev = np.std(trainX, axis=1)
train_mean = np.mean(trainX, axis=1)
train_median = np.median(trainX, axis=1)
train_datamean = np.mean(trainX, axis=1)
train_cfpmean = np.mean(trainX, axis=1)

val_std_dev = np.std(valX, axis=1)
val_mean = np.mean(valX, axis=1)
val_median = np.median(valX, axis=1)
val_datamean = np.mean(valX, axis=1)
val_cfpmean = np.mean(valX, axis=1)

test_std_dev = np.std(testX, axis=1)
test_mean = np.mean(testX, axis=1)
test_median = np.median(testX, axis=1)
test_datamean = np.mean(testX, axis=1)
test_cfpmean = np.mean(testX, axis=1)

# Add time domain features
train_stdsProb = np.sum(np.abs(trainX - train_mean.values.reshape(-1, 1)), axis=1) / trainX.shape[1]
val_stdsProb = np.sum(np.abs(valX - val_mean.values.reshape(-1, 1)), axis=1) / valX.shape[1]
test_stdsProb = np.sum(np.abs(testX - test_mean.values.reshape(-1, 1)), axis=1) / testX.shape[1]

train_stdsMean = train_std_dev / train_mean
val_stdsMean = val_std_dev / val_mean
test_stdsMean = test_std_dev / test_mean

train_outsprob = np.sum((trainX - train_mean.values.reshape(-1, 1)) ** 2, axis=1) / trainX.shape[1]
val_outsprob = np.sum((valX - val_mean.values.reshape(-1, 1)) ** 2, axis=1) / valX.shape[1]
test_outsprob = np.sum((testX - test_mean.values.reshape(-1, 1)) ** 2, axis=1) / testX.shape[1]

train_outsDev = train_std_dev / train_median
val_outsDev = val_std_dev / val_median
test_outsDev = test_std_dev / test_median

train_hfInd = np.max(trainX.iloc[:, :30], axis=1) - np.max(trainX.iloc[:, 30:60], axis=1)
train_hfAMp = np.sum(trainX.iloc[:, :30], axis=1) / np.sum(trainX.iloc[:, 30:60], axis=1)
train_bww99 = np.sum(trainX.iloc[:, :99], axis=1) / np.sum(trainX, axis=1)
train_flb = np.sum(trainX.iloc[:, :30], axis=1) / np.sum(trainX, axis=1)
train_fub = np.sum(trainX.iloc[:, 60:], axis=1) / np.sum(trainX, axis=1)
train_bwwpwr = np.sum(trainX.iloc[:, :99] ** 2, axis=1) / np.sum(trainX ** 2, axis=1)
train_bw50 = np.sum(trainX.iloc[:, :50], axis=1) / np.sum(trainX, axis=1)
train_fmed = np.median(trainX, axis=1)
train_fmean = np.mean(trainX, axis=1)

val_hfInd = np.max(valX.iloc[:, :30], axis=1) - np.max(valX.iloc[:, 30:60], axis=1)
val_hfAMp = np.sum(valX.iloc[:, :30], axis=1) / np.sum(valX.iloc[:, 30:60], axis=1)
val_bww99 = np.sum(valX.iloc[:, :99], axis=1) / np.sum(valX, axis=1)
val_flb = np.sum(valX.iloc[:, :30], axis=1) / np.sum(valX, axis=1)
val_fub = np.sum(valX.iloc[:, 60:], axis=1) / np.sum(valX, axis=1)
val_bwwpwr = np.sum(valX.iloc[:, :99] ** 2, axis=1) / np.sum(valX ** 2, axis=1)
val_bw50 = np.sum(valX.iloc[:, :50], axis=1) / np.sum(valX, axis=1)
val_fmed = np.median(valX, axis=1)
val_fmean = np.mean(valX, axis=1)

test_hfInd = np.max(testX.iloc[:, :30], axis=1) - np.max(testX.iloc[:, 30:60], axis=1)
test_hfAMp = np.sum(testX.iloc[:, :30], axis=1) / np.sum(testX.iloc[:, 30:60], axis=1)
test_bww99 = np.sum(testX.iloc[:, :99], axis=1) / np.sum(testX, axis=1)
test_flb = np.sum(testX.iloc[:, :30], axis=1) / np.sum(testX, axis=1)
test_fub = np.sum(testX.iloc[:, 60:], axis=1) / np.sum(testX, axis=1)
test_bwwpwr = np.sum(testX.iloc[:, :99] ** 2, axis=1) / np.sum(testX ** 2, axis=1)
test_bw50 = np.sum(testX.iloc[:, :50], axis=1) / np.sum(testX, axis=1)
test_fmed = np.median(testX, axis=1)
test_fmean = np.mean(testX, axis=1)

trainFeatX = np.stack([train_hfInd, train_hfAMp, train_bww99, train_flb, train_fub, train_bwwpwr, train_bw50, train_fmed, train_fmean, train_std_dev, train_mean, train_median, train_datamean,  train_cfpmean, train_stdsProb, train_stdsMean, train_outsprob, train_outsDev], axis=1)
testFeatX = np.stack([test_hfInd, test_hfAMp, test_bww99, test_flb, test_fub, test_bwwpwr, test_bw50, test_fmed, test_fmean, test_std_dev, test_mean, test_median, test_datamean, test_cfpmean, test_stdsProb, test_stdsMean, test_outsprob, test_outsDev], axis=1)
valFeatX = np.stack([val_hfInd, val_hfAMp, val_bww99, val_flb, val_fub, val_bwwpwr, val_bw50, val_fmed, val_fmean, val_std_dev, val_mean, val_median, val_datamean, val_cfpmean, val_stdsProb, val_stdsMean, val_outsprob, val_outsDev], axis=1)

In [8]:
valy = np.array(valy)
testy = np.array(testy)
trainy = np.array(trainy)

In [9]:
#input scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(trainX)
scaled_train = scaler.transform(trainX)
scaled_test = scaler.transform(testX)
scaled_val = scaler.transform(valX)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
# from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras_tuner.tuners import RandomSearch

In [11]:
import keras_tuner

In [12]:
# class EeHyperModel(keras_tuner.HyperModel):
#     def build(self, hp):
#         n_timesteps, n_features, n_outputs = trainX.shape[1], 1, trainy.shape[1]
#         model = Sequential()
#         model.add(LSTM(hp.Int('units_LSTM', 80, 220, step=30), input_shape=(n_timesteps,n_features)))
#         model.add(Dropout(hp.Choice('dropout', [0.2, 0.5])))

#         for i in range(hp.Int('layers', 1, 3)):
#             model.add(Dense(
#                 units=hp.Int('units_' + str(i), 30, 200, step=30),
#                 activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])
#             ))

#         model.add(Dense(n_outputs, activation='sigmoid'))
#         model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(
#             hp.Choice('learning_rate', values=[1e-2, 1e-4])),
#                       metrics=['accuracy'])

#         return model

#     def fit(self, hp, model, *args, **kwargs):
#         history = model.fit(
#             *args,
#             validation_split=0.2,
#             batch_size=hp.Int("batch_size", 32, 264, step=32),
#             epochs=50,
#             callbacks=[tf.keras.callbacks.EarlyStopping(
#                 monitor='val_loss',
#                 patience=5,
#                 restore_best_weights=True
#             )],
#             **kwargs,
#         )
#         return history



In [13]:
# class EeHyperModel(keras_tuner.HyperModel):
#     def build(self, hp):
#       n_timesteps, n_features, n_outputs = trainX.shape[1], 1, trainy.shape[1]
#       model = Sequential()
#       model.add(LSTM(hp.Int('units_LSTM', 80, 220, step=30), input_shape=(n_timesteps,n_features)))
#       model.add(Dropout(hp.Choice('dropout', [0.2, 0.5])))

#       for i in range(hp.Int('layers', 1, 3)):
#         model.add(Dense(
#           units=hp.Int('units_' + str(i), 30, 200, step=30),
#           activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])
#         ))

#       model.add(Dense(n_outputs, activation='sigmoid'))
#       model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(
#           hp.Choice('learning_rate',
#                     values=[1e-2, 1e-4])),
#                     metrics=['accuracy'])

#       return model

#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             batch_size=hp.Int("batch_size", 32, 264, step=32),
#             **kwargs,
#         )

In [14]:
# tuner = RandomSearch(
#     EeHyperModel(),
#     objective='val_accuracy',
#     max_trials=10,
#     executions_per_trial=1,
#     directory='my_dir',
#     project_name='rnn_lstm'
# )

In [15]:
# tuner.search_space_summary()

In [16]:
# Will stop training if the "val_loss" hasn't improved in 3 epochs.
# tuner.search(trainX, trainy, epochs=2,
            #  validation_data=(valX,valy))

In [17]:
# tuner.results_summary()

In [18]:
# import the necessary packages
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate

In [22]:
# train model
verbose, epochs, batch_size = 1, 15, 64
n_timesteps, n_features, n_outputs = trainX.shape[1], trainFeatX.shape[1], trainy.shape[1]

series_in = Input(shape=(n_timesteps, 1))
other_in = Input(shape=(n_features,)) # not a timeseries just a vector

# An example graph
lstm_out = LSTM(128)(series_in)
merged = concatenate([lstm_out, other_in])
out = Dense(n_outputs, activation='sigmoid')(merged)
model = Model([series_in, other_in], out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# model = Sequential()
# model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
# model.add(Dropout(0.2))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(n_outputs))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model fitting
model.fit(
	[trainX, trainFeatX], trainy,
	validation_data=([valX, valFeatX], valy),
	epochs=10, batch_size=64)
# model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
# evaluation
# _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

Epoch 1/10
734/734 [==============================] - 34s 43ms/step - loss: 104196820959232.0000 - accuracy: 0.6194 - val_loss: 608061685760.0000 - val_accuracy: 0.7334
Epoch 2/10
734/734 [==============================] - 31s 43ms/step - loss: 92494125596672.0000 - accuracy: 0.6793 - val_loss: 489848176640.0000 - val_accuracy: 0.6188
Epoch 3/10
734/734 [==============================] - 31s 42ms/step - loss: 74280704409600.0000 - accuracy: 0.7256 - val_loss: 371380289536.0000 - val_accuracy: 0.8508
Epoch 4/10
734/734 [==============================] - 31s 42ms/step - loss: 55663044591616.0000 - accuracy: 0.8442 - val_loss: 245207285760.0000 - val_accuracy: 0.8815
Epoch 5/10
734/734 [==============================] - 31s 42ms/step - loss: 35343118106624.0000 - accuracy: 0.8569 - val_loss: 117497806848.0000 - val_accuracy: 0.8818
Epoch 6/10
734/734 [==============================] - 33s 45ms/step - loss: 15059779584000.0000 - accuracy: 0.8755 - val_loss: 3069057171456.0000 - val_accurac

In [23]:
# Evaluate the model on the test data
# my_model = tuner.get_best_models(num_models=1)[0]
# print(my_model[0])
test_loss, test_accuracy = model.evaluate([testX, testFeatX], testy)

# # Print the test accuracy
print("Test accuracy:", test_accuracy)

459/459 [==============================] - 7s 16ms/step - loss: 698780811264.0000 - accuracy: 0.8880
Test accuracy: 0.888018012046814


In [24]:
val_loss, val_accuracy = model.evaluate([valX, valFeatX], valy)

# # Print the val accuracy
print("Val accuracy:", val_accuracy)

367/367 [==============================] - 5s 14ms/step - loss: 16428092620800.0000 - accuracy: 0.8938
Val accuracy: 0.8938490152359009


In [25]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score

y_pred_test = model.predict([testX, testFeatX])
y_true_test = testy

y_pred_val = model.predict([valX, valFeatX])
y_true_val = valy

roc_auc = roc_auc_score(y_true_test, y_pred_test)
print('Model: ROC AUC=%.3f' % (roc_auc))

367/367 [==============================] - 5s 13ms/step
Model: ROC AUC=0.905


In [26]:
roc_auc_val = roc_auc_score(y_true_val, y_pred_val)
print('Model: ROC AUC VAL=%.3f' % (roc_auc))

Model: ROC AUC VAL=0.905


In [27]:
y_pred_val = np.array(y_pred_val > 0.5)

# validation %
precision_recall_fscore_support(y_true_val, y_pred_val, average=None, labels=[True, False])

(array([0.87981636, 0.90902642]),
 array([0.91274026, 0.87489333]),
 array([0.89597596, 0.89163333]),
 array([5879, 5859]))

In [28]:
y_pred_test = np.array(y_pred_test > 0.5)

# test %
precision_recall_fscore_support(y_true_test, y_pred_test, average=None, labels=[True, False])

(array([0.86991658, 0.90785714]),
 array([0.91187321, 0.86427309]),
 array([0.89040091, 0.88552916]),
 array([7319, 7353]))